# Jupyter Notebook to train a model

In [16]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tf_explain.callbacks.grad_cam import GradCAMCallback

# use random seed to reproduce results
np.random.seed(42)
tf.random.set_seed(42)

### lists possible devices (CPU, GPU), used to check if GPU is recognized/exists

In [17]:
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Methods

### used to save a trained model as a json file and its weights as a h5 file

In [18]:
def save_model(model, model_name):
    my_model = model.to_json()
    with open(f'./saved_models/{model_name}.json', "w") as file:
        file.write(my_model)
    # serialize weights to HDF5
    model.save_weights(f'./saved_models/{model_name}_weights.h5')

### used to build the base model using predefined architectures
currently: vgg16, xception, resnet

In [19]:
def build_base_model(architecture, weights):
    input = tf.keras.Input(shape=(224, 224, 3))
    if architecture == 'vgg16':
        return tf.keras.applications.vgg16.VGG16(weights=weights, include_top=False, input_tensor=input)
    if architecture == 'xception':
        return tf.keras.applications.xception.Xception(weights=weights, include_top=False, input_tensor=input)
    if architecture == 'resnet':
        return tf.keras.applications.resnet.ResNet50(weights=weights, include_top=False, input_tensor=input)

### gets base model as input and builds a new top layer and returns the model with custom top layers

In [20]:
def build_model(base_model):
    flat = keras.layers.Flatten(name='flatten')(base_model.output)
    dense_1 = keras.layers.Dense(2800)(flat)
    dropout = keras.layers.Dropout(0.25)(dense_1)
    batch = keras.layers.BatchNormalization()(dropout)
    output = keras.layers.Dense(1, activation='sigmoid')(batch)
    return tf.keras.Model(base_model.input, output)

### gets a model as input and returns a model compiled with the adam optimizer

In [21]:
def compile_model(model, alpha, beta1, beta2, metrics):
    optimizer = keras.optimizers.Adam(learning_rate=alpha, beta_1=beta1, beta_2=beta2)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=metrics)
    return model

### gets a model as input and trains it on the data-set with the defined callbacks and epochs

In [22]:
def train_model(model, train_set, validation_set, epochs, callback):
    return model.fit(train_set,
                     validation_data=validation_set,
                     epochs=epochs,
                     callbacks=callback)

### gets a model as input and changes its layers trainable attribute

In [23]:
def set_layers_trainable(trainable, input_model):
    for layer in input_model.layers:
        layer.trainable = trainable

## Data preprocessing

#### preprocessing of the images applied when loading image data set from disk with tensorflows flow_from_directory


In [24]:
image_gen = keras.preprocessing.image.ImageDataGenerator(rotation_range=20,  # rotate the image 20 degrees
                                                         width_shift_range=0.2,
                                                         height_shift_range=0.2,
                                                         rescale=1 / 255,  # Rescale the image by normalzing it.
                                                         shear_range=0.15,
                                                         # Shear means cutting away part of the image (max 20%)
                                                         zoom_range=0.15,  # Zoom in by 15% max
                                                         horizontal_flip=True,  # Allow horizontal flipping
                                                         fill_mode='nearest'
                                                         # Fill in missing pixels with the nearest filled value
                                                         )

### path to the data set

In [25]:
train_data_path = './data/data_full/train'  #local notebook
test_data_path = './data/data_full/test'  #local notebook
validation_data_path = './data/data_full/val'  #local notebook

### generate training set by loading the images from their directories with flow_from_directory
### important: the folder structure has to match! i.e {train} -> {ok,def}
### at the "same time" the data augmentation is applied on the images through the ImageDataGenerator

In [26]:
batch_size = 12
train_image_gen = image_gen.flow_from_directory(train_data_path,
                                                target_size=(224, 224),
                                                batch_size=batch_size,
                                                class_mode='binary')

Found 48271 images belonging to 2 classes.


In [27]:
valid_image_gen = image_gen.flow_from_directory(validation_data_path,
                                                target_size=(224, 224),
                                                batch_size=batch_size,
                                                class_mode='binary')

Found 6033 images belonging to 2 classes.


In [28]:
test_image_gen = image_gen.flow_from_directory(test_data_path,
                                               target_size=(224, 224),
                                               batch_size=batch_size,
                                               class_mode='binary')

Found 6036 images belonging to 2 classes.


# Training of the model

### Pre Training
* build base model with vgg16 architecture and pretrained with imagenet weights
* add custom layers on base model
* compile model with adam optimizer
* freeze layers of pretrained vgg16 base model to not destroy imagenet weights
* train model

In [14]:
vgg16 = build_base_model('vgg16', 'imagenet')
model = build_model(vgg16)
model = compile_model(model, 8.837e-05, 0.9, 0.999, ['accuracy', 'Recall', 'Precision', 'AUC'])
set_layers_trainable(False, vgg16)
history = train_model(model, train_image_gen, valid_image_gen, 5, [])

2021-12-15 14:37:15.720107: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-15 14:37:15.720502: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 14:37:15.720673: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.8225GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2021-12-15 14:37:15.720710: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library li

Epoch 1/5


2021-12-15 14:37:18.514117: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-12-15 14:37:18.822378: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-12-15 14:37:19.752241: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2021-12-15 14:37:19.775332: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


4023/4023 [==============================] - 739s 181ms/step - loss: 0.3307 - accuracy: 0.8598 - recall: 0.7485 - precision: 0.5981 - auc: 0.8971 - val_loss: 0.3369 - val_accuracy: 0.8850 - val_recall: 0.3053 - val_precision: 1.0000 - val_auc: 0.9604
Epoch 2/5
4023/4023 [==============================] - 754s 188ms/step - loss: 0.0840 - accuracy: 0.9703 - recall: 0.8938 - precision: 0.9253 - auc: 0.9897 - val_loss: 0.0278 - val_accuracy: 0.9874 - val_recall: 0.9279 - val_precision: 0.9957 - val_auc: 0.9997
Epoch 3/5
4023/4023 [==============================] - 737s 183ms/step - loss: 0.0565 - accuracy: 0.9806 - recall: 0.9282 - precision: 0.9524 - auc: 0.9934 - val_loss: 0.0262 - val_accuracy: 0.9924 - val_recall: 0.9910 - val_precision: 0.9640 - val_auc: 0.9997
Epoch 4/5
4023/4023 [==============================] - 754s 187ms/step - loss: 0.0303 - accuracy: 0.9899 - recall: 0.9639 - precision: 0.9747 - auc: 0.9980 - val_loss: 0.2309 - val_accuracy: 0.9130 - val_recall: 1.0000 - val_pr

In [15]:
save_model(model, 'vgg16_only_pretrained_tuned_parameters')

### Full Training
* un freeze layers of pretrained vgg16 base model
* define callbacks
    * tensorboard callback to generate reports which can be viewed in tensorboard
    * early stopping callback to stop training after monitored metric has not changed after, by the patience defined, epochs
* compile model with adam optimizer
* train model

In [18]:
set_layers_trainable(True, vgg16)

#### Callbacks

In [19]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=5, write_graph=True, write_images=True)
custom_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

2021-12-07 20:07:19.281741: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-12-07 20:07:19.281774: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-12-07 20:07:19.281993: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1365] Profiler found 1 GPUs
2021-12-07 20:07:19.290065: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcupti.so.10.1
2021-12-07 20:07:19.391355: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2021-12-07 20:07:19.392395: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [20]:
model = compile_model(model, 0.0001, 0.9, 0.999, ['accuracy', 'Recall', 'Precision', 'AUC'])
history = train_model(model, train_image_gen, valid_image_gen, 100,
                      [custom_early_stopping, tensorboard_callback])

Epoch 1/100
  1/800 [..............................] - ETA: 47:38 - loss: 3.3124e-04 - accuracy: 1.0000 - recall: 1.0000 - precision: 1.0000 - auc: 1.0000

2021-12-07 20:07:25.043807: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-12-07 20:07:25.043830: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-12-07 20:07:25.044275: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES


  2/800 [..............................] - ETA: 5:07 - loss: 0.0011 - accuracy: 1.0000 - recall: 1.0000 - precision: 1.0000 - auc: 1.0000     

2021-12-07 20:07:25.307700: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-12-07 20:07:25.318480: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:228]  GpuTracer has collected 0 callback api events and 0 activity events. 
2021-12-07 20:07:25.322665: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-12-07 20:07:25.329030: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/train/plugins/profile/2021_12_07_20_07_25
2021-12-07 20:07:25.330457: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to ./logs/train/plugins/profile/2021_12_07_20_07_25/pop-os.trace.json.gz
2021-12-07 20:07:25.362010: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/train/plugins/profile/2021_12_07_20_07_25
2021-12-07 20:07:25.366774: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped

800/800 [==============================] - 199s 244ms/step - loss: 0.0358 - accuracy: 0.9899 - recall: 0.9880 - precision: 0.9919 - auc: 0.9979 - val_loss: 0.0393 - val_accuracy: 0.9880 - val_recall: 0.9900 - val_precision: 0.9860 - val_auc: 0.9977
Epoch 2/100
800/800 [==============================] - 194s 243ms/step - loss: 0.0353 - accuracy: 0.9904 - recall: 0.9912 - precision: 0.9896 - auc: 0.9982 - val_loss: 0.0273 - val_accuracy: 0.9940 - val_recall: 1.0000 - val_precision: 0.9881 - val_auc: 0.9989
Epoch 3/100
800/800 [==============================] - 195s 243ms/step - loss: 0.0481 - accuracy: 0.9873 - recall: 0.9888 - precision: 0.9859 - auc: 0.9972 - val_loss: 0.0890 - val_accuracy: 0.9810 - val_recall: 0.9619 - val_precision: 1.0000 - val_auc: 0.9959
Epoch 4/100
800/800 [==============================] - 195s 244ms/step - loss: 0.0164 - accuracy: 0.9933 - recall: 0.9942 - precision: 0.9926 - auc: 0.9997 - val_loss: 0.0294 - val_accuracy: 0.9910 - val_recall: 0.9920 - val_prec

# Evaluation of the model on the test set

In [21]:
loss, accuracy, recall, precision, auc = model.evaluate(test_image_gen)

100/101 [============================>.] - ETA: 0s - loss: 0.0058 - accuracy: 0.9990 - recall: 0.9980 - precision: 1.0000 - auc: 1.0000

2021-12-07 22:26:28.850816: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.40GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


101/101 [==============================] - 12s 122ms/step - loss: 0.0058 - accuracy: 0.9990 - recall: 0.9980 - precision: 1.0000 - auc: 1.0000


In [22]:
# F1 score
f1 = 2 * ((precision * recall) / (precision + recall))

print(f"loss: {loss}, \n"
      f"accuracy: {accuracy}, \n"
      f"recall: {recall}, \n"
      f"precision: {precision}, \n"
      f"auc: {auc}, \n"
      f"F1: {f1}")

loss: 0.00580719206482172, 
accuracy: 0.9990010261535645, 
recall: 0.9980040192604065, 
precision: 1.0, 
auc: 0.9999960064888, 
F1: 0.9990010126504488


# Saving the model to use it for predictions, heatmaps, etc.

In [23]:
save_model(model, 'xception_heavily_reduced_data')